In [1]:
%load_ext autoreload
%autoreload 2

from NewsContent import *
from UserContent import *
from preprocessing import *
from PEGenerator import *
import PEGenerator
from models import *
from utils import *
from Encoders import *

import os
import numpy as np
import json
import random

In [2]:
data_root_path = "../data/Challenge/"
embedding_path = None
KG_root_path = None
popularity_path = "../data/Challenge/popularity"
config = {'title_length':30,
              'body_length':100,
              'max_clicked_news':50,
              'npratio':1,
              'news_encoder_name':"CNN",
              'user_encoder_name':"Att",
             'attrs':['title','vert','entity'],
             'word_filter':0,
             'data_root_path':data_root_path,
             'embedding_path':embedding_path,
             'KG_root_path':KG_root_path,
            'popularity_path':popularity_path,
             'max_entity_num':5}

In [7]:
News = NewsContent(config)
# TrainUsers = UserContent(News.news_index,config,'train.tsv',2)
# ValidUsers = UserContent(News.news_index,config,'val.tsv',1)
# TestUsers = UserContent(News.news_index,config,'test.tsv',2)

['9774516', '93', '1', '34']
oof
['9771051', '93', '4', '52']
oof
['9770028', '93', '1', '42']
oof
['9775402', '93', '6', '52']
oof
['9774461', '93', '12', '46']
oof
['9759544', '93', '9', '43']
oof
['9773947', '93', '5', '48']
oof
['9142581', '93', '1', '6']
oof
['9775331', '93', '4', '36']
oof
['9775371', '93', '1', '52']
oof
['9759966', '93', '5', '34']
oof
['9778669', '151', '5', '87']
oof
['9778736', '151', '6', '56']
oof
['9778623', '151', '14', '104']
oof
['9089120', '151', '0', '3']
oof
['9778661', '151', '14', '69']
oof
['9777492', '151', '9', '84']
oof
['9778718', '151', '6', '78']
oof
['9778657', '151', '18', '95']
oof
['9778682', '151', '10', '87']
oof
['9482970', '151', '0', '3']
oof
['9718262', '151', '0', '1']
oof
['9718298', '151', '0', '2']
oof
['9778728', '151', '2', '50']
oof
['9080070', '151', '0', '3']
oof
['9420172', '151', '0', '2']
oof
['9717914', '151', '0', '2']
oof
['9777397', '151', '5', '71']
oof
['9778369', '151', '3', '30']
oof
['9777856', '151', '6', '28

In [8]:
print(News.news_index)

{'N3037230': 1, 'N3044020': 2, 'N3057622': 3, 'N3073151': 4, 'N3193383': 5, 'N3196611': 6, 'N3200325': 7, 'N3200913': 8, 'N3209311': 9, 'N3209357': 10, 'N3223114': 11, 'N3251664': 12, 'N3971783': 13, 'N3977720': 14, 'N3983641': 15, 'N3987418': 16, 'N3988589': 17, 'N3993395': 18, 'N3994264': 19, 'N3997229': 20, 'N3997233': 21, 'N3998680': 22, 'N4000567': 23, 'N4000823': 24, 'N4003163': 25, 'N4003362': 26, 'N4003431': 27, 'N4003926': 28, 'N4007811': 29, 'N4008213': 30, 'N4009177': 31, 'N4014381': 32, 'N4021392': 33, 'N4024289': 34, 'N4024449': 35, 'N4026960': 36, 'N4035059': 37, 'N4035605': 38, 'N4040731': 39, 'N4040737': 40, 'N4043121': 41, 'N4044042': 42, 'N4045439': 43, 'N4048845': 44, 'N4050800': 45, 'N4051858': 46, 'N4055236': 47, 'N4056868': 48, 'N4062565': 49, 'N4066000': 50, 'N4067703': 51, 'N4067935': 52, 'N4069792': 53, 'N4073764': 54, 'N4074883': 55, 'N4081011': 56, 'N4081857': 57, 'N4083620': 58, 'N4084225': 59, 'N4085973': 60, 'N4086433': 61, 'N4091672': 62, 'N4092282': 63, 

In [50]:
print(len(News.news_stat_imp))
print(sum(sum(News.news_stat_imp)))

11778
0.0


In [31]:
train_sess,train_buckets, train_user_id, train_label = get_train_input(TrainUsers.session,News.news_index,config)
test_impressions, test_userids = get_test_input(TestUsers.session,News.news_index)
val_impressions, val_userids = get_test_input(ValidUsers.session,News.news_index)

19779
22179
2473
2472


In [ ]:
# title_word_embedding_matrix, have_word = load_matrix(embedding_path,News.word_dict)

In [ ]:
train_generator = TrainGenerator(News,TrainUsers,train_sess,train_user_id,train_buckets,train_label,32)
test_user_generator = UserGenerator(News,TestUsers,32)
val_user_generator = UserGenerator(News,ValidUsers,32)
news_generator = NewsGenerator(News,32)

In [ ]:
# for i in range(10):
#     model_config = {
#         'news_encoder':1,
#         'popularity_user_modeling':True,
#         'rel':True,
#         'ctr':True,
#         'content':True,
#         'rece_emb':True,
#         'activity':True
# 
#     }
# 
#     model,user_encoder,news_encoder,bias_news_encoder,bias_content_scorer,scaler,time_embedding_layer,activity_gater = create_pe_model(config,model_config,News,title_word_embedding_matrix,News.entity_embedding)
#     model.fit_generator(train_generator,epochs=2)
#     news_scoring = news_encoder.predict_generator(news_generator,verbose=True)
#     user_scoring = user_encoder.predict_generator(test_user_generator,verbose=True)
#     val_user_scoring = user_encoder.predict_generator(val_user_generator,verbose=True)
# 
# 
#     news_bias_vecs = bias_news_encoder.predict_generator(news_generator,verbose=True)
# 
#     if model_config['content'] and not model_config['rece_emb']:
#         bias_candidate_score = bias_content_scorer.predict(news_bias_vecs,batch_size=32,verbose=True)
#         bias_candidate_score = bias_candidate_score[:,0]
#     else:
#         bias_candidate_score = 0
# 
#     ctr_weight = scaler.get_weights()[0][0,0]
#     time_embedding_matrix = time_embedding_layer.get_weights()[0]
#     
#     predicted_activity_gates = activity_gater.predict(user_scoring,verbose=True)
#     predicted_activity_gates = predicted_activity_gates[:,0]
#     
#     val_predicted_activity_gates = activity_gater.predict(val_user_scoring,verbose=True)
#     val_predicted_activity_gates = val_predicted_activity_gates[:,0]
#     
#     rankings = news_ranking(model_config,ctr_weight,predicted_activity_gates,user_scoring,news_scoring,
#                                 bias_candidate_score,news_bias_vecs,time_embedding_matrix,bias_content_scorer,
#                                 News,test_impressions)
#     
#     val_rankings = news_ranking(model_config,ctr_weight,val_predicted_activity_gates,val_user_scoring,news_scoring,
#                                bias_candidate_score,news_bias_vecs,time_embedding_matrix,bias_content_scorer,
#                                News,val_impressions)
#     
#     performance = evaluate_performance(rankings,test_impressions)
#     val_performance = evaluate_performance(val_rankings,val_impressions)
# 
#     cold = []
#     for TOP_COLD_NUM in [0,1,3,5,]:
#         g = evaluate_cold_users(rankings,test_impressions,TestUsers.click,TOP_COLD_NUM)
#         cold.append(g)
#     diversity = []
#     for TOP_DIVERSITY_NUM in range(1,11):
#         div_top = evaluate_diversity_topic_all(TOP_DIVERSITY_NUM,rankings,test_impressions,News,TestUsers)
#         div_ilxd = evaluate_density_ILxD(TOP_DIVERSITY_NUM,rankings,test_impressions,news_scoring)
#         diversity.append([div_top,div_ilxd])
